In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#1. Import all packages

import pandas as pd
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import math
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import multilabel_confusion_matrix
from statistics import mean

In [4]:
#2. Load dataset 
data = pd.read_csv("data.csv",header=0)

#delete unused columns
data.drop(["Unnamed: 32", "id"], axis=1, inplace=True)

#change label M = malignant = 1 dan B = Benign = 0
data.diagnosis = [1 if each == "M" else 0 for each in data.diagnosis]
m_data = data.loc[data['diagnosis'] == 1] 
b_data = data.loc[data['diagnosis'] == 0]

b_data.head(10)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
19,0,13.540,14.36,87.46,566.3,0.09779,0.08129,0.06664,0.047810,0.1885,...,15.110,19.26,99.70,711.2,0.14400,0.17730,0.23900,0.12880,0.2977,0.07259
20,0,13.080,15.71,85.63,520.0,0.10750,0.12700,0.04568,0.031100,0.1967,...,14.500,20.49,96.09,630.5,0.13120,0.27760,0.18900,0.07283,0.3184,0.08183
21,0,9.504,12.44,60.34,273.9,0.10240,0.06492,0.02956,0.020760,0.1815,...,10.230,15.66,65.13,314.9,0.13240,0.11480,0.08867,0.06227,0.2450,0.07773
37,0,13.030,18.42,82.61,523.8,0.08983,0.03766,0.02562,0.029230,0.1467,...,13.300,22.81,84.46,545.9,0.09701,0.04619,0.04833,0.05013,0.1987,0.06169
46,0,8.196,16.84,51.71,201.9,0.08600,0.05943,0.01588,0.005917,0.1769,...,8.964,21.96,57.26,242.2,0.12970,0.13570,0.06880,0.02564,0.3105,0.07409
48,0,12.050,14.63,78.04,449.3,0.10310,0.09092,0.06592,0.027490,0.1675,...,13.760,20.70,89.88,582.6,0.14940,0.21560,0.30500,0.06548,0.2747,0.08301
49,0,13.490,22.30,86.91,561.0,0.08752,0.07698,0.04751,0.033840,0.1809,...,15.150,31.82,99.00,698.8,0.11620,0.17110,0.22820,0.12820,0.2871,0.06917
50,0,11.760,21.60,74.72,427.9,0.08637,0.04966,0.01657,0.011150,0.1495,...,12.980,25.72,82.98,516.5,0.10850,0.08615,0.05523,0.03715,0.2433,0.06563
51,0,13.640,16.34,87.21,571.8,0.07685,0.06059,0.01857,0.017230,0.1353,...,14.670,23.19,96.08,656.7,0.10890,0.15820,0.10500,0.08586,0.2346,0.08025
52,0,11.940,18.24,75.71,437.6,0.08261,0.04751,0.01972,0.013490,0.1868,...,13.100,21.33,83.67,527.2,0.11440,0.08906,0.09203,0.06296,0.2785,0.07408


In [5]:
#3. Share to test and train data

x = data.iloc[:, 1:]
y = data['diagnosis'].tolist()

#share test and train data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [6]:
#4. Train random forest algorithm

model_clf = RandomForestClassifier(n_estimators=500, max_depth=2, random_state=0)
model_clf.fit(x_train, y_train)

RandomForestClassifier(max_depth=2, n_estimators=500, random_state=0)

In [7]:
#5. apply the trained learner to test new data

#to make predicition of test data
y_pred = model_clf.predict(x_test)

In [8]:
#6. confusion matrix

#actual and predicted classes
lst_actual_class = y_test
lst_predicted_class = y_pred
lst_classes = [0, 1]

#confusion matrix
tn, fp, fn, tp = confusion_matrix(lst_actual_class, lst_predicted_class, labels=lst_classes).ravel()
sensitivity = round(tp/(tp+fn), 3);
specificity = round(tn/(tn+fp), 3);
accuracy = round((tp+tn)/(tp+fp+tn+fn), 3);
balanced_accuracy = round((sensitivity+specificity)/2, 3);
mcc = round(((tp*tn)-(fp*fn))/(math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))), 3);
precision = round(tp/(tp+fp), 3);
f1_score = round((2*tp)/((2*tp)+fp+fn), 3);

print("TP={0}, FP={1}, TN={2}, FN={3}".format(tp, fp, tn, fn));
print("Sensitivity: {0}".format(sensitivity));
print("Specificity: {0}".format(specificity));
print("Accuracy: {0}".format(accuracy));
print("Balanced Accuracy: {0}".format(balanced_accuracy));
print("MCC: {0}".format(mcc));
print("Precision:{0}".format(precision));
print("F1_Score:{0}".format(f1_score));

TP=51, FP=1, TN=88, FN=3
Sensitivity: 0.944
Specificity: 0.989
Accuracy: 0.972
Balanced Accuracy: 0.966
MCC: 0.94
Precision:0.981
F1_Score:0.962
